In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import csv

In [ ]:
transaction = pd.read_csv("transactions.csv", delimiter="|")
items = pd.read_csv("items.csv", delimiter="|")

In [ ]:
transaction.head()

In [ ]:
print(transaction.shape)
print(items.shape)

In [ ]:
data = pd.merge(items, transaction, on=["itemID"])

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
# choosing Target variable
#print(inner_merged_total['basket'].value_counts())
#print(inner_merged_total['click'].value_counts())
print(data['order'].value_counts())

In [ ]:
data.head()
#print(inner_merged_total['Interest'].value_counts())

In [ ]:
##generate frequency
test = data.value_counts(['itemID']).reset_index(name='Frequency')
print(test.head())

In [ ]:
#Merging frequency 
book_data = pd.merge(data, test, on=["itemID"])
book_data.head()

In [ ]:
#remove dupicate rows 
print(book_data.shape)
# book_data.drop_duplicates(subset ="itemID", keep='first' ,inplace=True)
# print(book_data.shape)

In [ ]:
book_data["itemID"].value_counts()

In [ ]:
book_data.head()

In [ ]:
book_rating= book_data.copy()
book_rating = book_rating.drop(['sessionID','author','publisher'], axis=1)

In [ ]:
import numpy as np
book_rating['log_frequency'] = np.log(book_data['Frequency'])

In [ ]:
print(book_rating['log_frequency'].value_counts())

In [ ]:
book_rating['basket'].value_counts()

In [ ]:
bin_labels_5 = ['E','D','C', 'B', 'A']
book_rating['Grade'] = pd.qcut(book_rating['log_frequency'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=bin_labels_5, duplicates='drop')
book_rating


In [ ]:
book_rating.drop_duplicates(subset ="title", keep='first' ,inplace=True)

In [ ]:
book_rating.reset_index()

In [ ]:
##data cleaning
values = {'basket': 0, 'order': 0,'click':0}
book_rating = book_rating.fillna(value=values)

In [ ]:
#calculate relevance score

def relevance(book_rating):
    if (book_rating['order'] != 0.0 and book_rating['basket'] != 0.0):
        return "A"
    elif (book_rating['order'] != 0.0 and  book_rating['basket'] == 0.0):
        return "B"
    elif (book_rating['order'] == 0.0 and  book_rating['basket'] != 0.0):
        return "C"
    else:
        return "D"
    

book_rating['relevance_score'] = book_rating.apply(relevance, axis = 1)


# Split data based on Frequency

In [ ]:
##relevant book
book_rating_G = book_rating[book_rating['relevance_score'] != "D"]
print(book_rating_G.shape)

## books arent transacted pt 1
book_rating_B1 = book_rating[book_rating['relevance_score'] == "D"]
book_rating_B1 = book_rating_B1.head(34602)
print(book_rating_B1.shape)

## books arent transacted pt 2
book_rating_B2 = book_rating_B1.tail(34602)
print(book_rating_B2.shape)

# Matrix part 1

In [ ]:
#creating combined target feature
def parseFeatureA(book_rating):
    data = str(book_rating["main topic"]) + ','
    data += str(book_rating["subtopics"]) + ','
    data += str(book_rating["Grade"]) + ','
    data += str(book_rating["relevance_score"])
    x = data.replace('[','')
    y = x.replace(']','')
    return y

In [ ]:
book_rating['parsedFeatureStr'] = book_rating.apply(parseFeatureA, axis = 1)

In [ ]:
book_rating.head(5)

In [ ]:
book_rating = book_rating.reset_index(drop=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer



vectorizer = TfidfVectorizer(min_df=3, max_features=None,
                            strip_accents='unicode', analyzer='word',token_pattern=r',',ngram_range=(1,3),stop_words='english')


In [ ]:
parsedFeatureMatrix = vectorizer.fit_transform(book_rating['parsedFeatureStr'])

In [ ]:
parsedFeatureMatrix

In [ ]:
parsedFeatureMatrix.shape

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel 
from sklearn.metrics.pairwise import cosine_similarity 

sig = sigmoid_kernel(parsedFeatureMatrix, parsedFeatureMatrix)
# cos = cosine_similarity(parsedFeatureMatrix, parsedFeatureMatrix)

In [ ]:
indices = pd.Series(book_rating.index, index = book_rating['title'])

In [ ]:
indices_df = pd.Series(book_rating.index, index = book_rating['title']).to_frame()

In [ ]:
indices_itemID_mapping = pd.merge(indices_df, book_rating, on=["title"])

In [ ]:
indices_itemID_mapping = indices_itemID_mapping.drop(['index'], axis=1)

In [ ]:
indices_itemID_mapping
# print(indices_df)

In [ ]:
def recommender(title, sig=sig):
    index = indices[title]
    print(index)
    
    # get similarity scores
    sig_scores = list(enumerate(sig[index]))
    #sort by scores
    sig_scores = sorted(sig_scores, key=lambda x:x[1], reverse=True)
    sig_scores = sig_scores[1:6]
    
    #relate to book
    book_indices = [i[0] for i in sig_scores]
    
    return book_rating['itemID'].iloc[book_indices]
    
    
    

In [ ]:
recommender_df = recommender('Red Queen 1').to_frame()

In [ ]:
recommender_df

## Populate evaluation DF Part 1


In [ ]:
## Use Evaluation dataset to recommend 5 itemID

evaluation_df = pd.read_csv("evaluation.csv", delimiter="|")

evaluation_df

In [ ]:
title_df = book_rating.drop(['basket','order','main topic','subtopics','log_frequency','Grade', 'relevance_score', 'click','Frequency'], axis=1)
title_df

In [ ]:
submission_df = pd.merge(evaluation_df, title_df, on=["itemID"]) 
# submission_df_p1 = submission_df_p1.drop(['author','publisher','main topic','subtopics'], axis=1)
# submission_df_p1 = submission_df_p1.drop('index')
submission_df.reset_index(drop=True, inplace=True)

In [ ]:
submission_df = submission_df.drop("parsedFeatureStr", axis=1)
submission_df

In [ ]:
def rec(submission_df, num):
    recommended_books = []
    recommender_df = recommender(submission_df['title']).to_frame()
    recommender_df = recommender_df.reset_index(drop=True)
    for ind,row in recommender_df.iterrows():
        recommended_books.append(row['itemID'])
        
    return recommended_books[num]

In [ ]:
recommend = ['rec1','rec2','rec3','rec4','rec5']
for i in range(0,5):
    key = recommend[i]
    submission_df[key] = submission_df.apply(rec,num=i, axis=1)
    

In [ ]:
submission_df.head()

In [ ]:
submission_df = submission_df.drop("title", axis=1)

In [ ]:
Export_csv_p1 = submission_df.to_csv("final_submission_df_p1.csv", index=False)

## Matrix Part 2

In [ ]:
#creating combined target feature
def parseFeatureB1(book_rating_B1):
    data = str(book_rating_B1["main topic"]) + ','
    data += str(book_rating_B1["subtopics"]) + ','
    data += str(book_rating_B1["Grade"]) + ','
    data += str(book_rating_B1["relevance_score"])
    x = data.replace('[','')
    y = x.replace(']','')
    return y

In [ ]:
book_rating_B1.head(5)

In [ ]:
book_rating_B1['parsedFeatureStr'] = book_rating_B1.apply(parseFeatureB1, axis = 1)

In [ ]:
book_rating_B1 = book_rating_B1.reset_index(drop=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer



vectorizer = TfidfVectorizer(min_df=3, max_features=None,
                            strip_accents='unicode', analyzer='word',token_pattern=r',',ngram_range=(1,3),stop_words='english')

In [ ]:
parsedFeatureMatrix = vectorizer.fit_transform(book_rating_B1['parsedFeatureStr'])

In [ ]:
parsedFeatureMatrix.shape

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel 
from sklearn.metrics.pairwise import cosine_similarity 

sig = sigmoid_kernel(parsedFeatureMatrix, parsedFeatureMatrix)

In [ ]:
indices = pd.Series(book_rating_B1.index, index = book_rating_B1['title'])

In [ ]:
indices_df = pd.Series(book_rating_B1.index, index = book_rating_B1['title']).to_frame()

In [ ]:
indices_itemID_mapping = pd.merge(indices_df, book_rating_B1, on=["title"])

In [ ]:
indices_itemID_mapping = indices_itemID_mapping.drop(['index'], axis=1)

In [ ]:
indices_itemID_mapping

In [ ]:
def recommender(title, sig=sig):
    index = indices[title]
    print(index)
    
    # get similarity scores
    sig_scores = list(enumerate(sig[index]))
    #sort by scores
    sig_scores = sorted(sig_scores, key=lambda x:x[1], reverse=True)
    sig_scores = sig_scores[1:6]
    
    #relate to book
    book_indices = [i[0] for i in sig_scores]
    
    return book_rating['itemID'].iloc[book_indices]

In [ ]:
recommender_df = recommender('Einfach zeichnen! Step by Step').to_frame()

In [ ]:
recommender_df

## Populate evaluation DF Part 2

In [ ]:
evaluation_df

In [ ]:
title_df = book_rating_B1.drop(['basket','order','main topic','subtopics','log_frequency','Grade', 'relevance_score', 'click','Frequency'], axis=1)
title_df

In [ ]:
submission_df_p2 = pd.merge(evaluation_df, title_df, on=["itemID"]) 
# submission_df_p1 = submission_df_p1.drop(['author','publisher','main topic','subtopics'], axis=1)
# submission_df_p1 = submission_df_p1.drop('index')
submission_df_p2.reset_index(drop=True, inplace=True)

In [ ]:
def rec2(submission_df_p2, num):
    recommended_books = []
    recommender_df = recommender(submission_df_p2['title']).to_frame()
    recommender_df = recommender_df.reset_index(drop=True)
    for ind,row in recommender_df.iterrows():
        recommended_books.append(row['itemID'])
        
    return recommended_books[num]

In [ ]:
recommend = ['rec1','rec2','rec3','rec4','rec5']
for i in range(0,5):
    key = recommend[i]
    submission_df_p2[key] = submission_df_p2.apply(rec2,num=i, axis=1)

In [ ]:
submission_df_p2 = submission_df_p2.drop(['title','parsedFeatureStr'], axis=1)

In [ ]:
submission_df_p2

In [ ]:
Export_csv_p2 = submission_df_p2.to_csv("submission_df_p2.csv", index=False)

## Matrix part 3

In [ ]:
#creating combined target feature
def parseFeatureB2(book_rating_B2):
    data = str(book_rating_B2["main topic"]) + ','
    data += str(book_rating_B2["subtopics"]) + ','
    data += str(book_rating_B2["Grade"]) + ','
    data += str(book_rating_B2["relevance_score"])
    x = data.replace('[','')
    y = x.replace(']','')
    return y

In [ ]:
book_rating_B2.head(5)

In [ ]:
book_rating_B2['parsedFeatureStr'] = book_rating_B2.apply(parseFeatureB2, axis = 1)

In [ ]:
book_rating_B2 = book_rating_B2.reset_index(drop=True)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer



vectorizer = TfidfVectorizer(min_df=3, max_features=None,
                            strip_accents='unicode', analyzer='word',token_pattern=r',',ngram_range=(1,3),stop_words='english')

In [ ]:
parsedFeatureMatrix = vectorizer.fit_transform(book_rating_B2['parsedFeatureStr'])

In [ ]:
parsedFeatureMatrix.shape

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel 
from sklearn.metrics.pairwise import cosine_similarity 

sig = sigmoid_kernel(parsedFeatureMatrix, parsedFeatureMatrix)

In [ ]:
indices = pd.Series(book_rating_B2.index, index = book_rating_B2['title'])

In [ ]:
indices_df = pd.Series(book_rating_B2.index, index = book_rating_B2['title']).to_frame()

In [ ]:
indices_itemID_mapping = pd.merge(indices_df, book_rating_B2, on=["title"])

In [ ]:
indices_itemID_mapping = indices_itemID_mapping.drop(['index'], axis=1)

In [ ]:
indices_itemID_mapping

In [ ]:
def recommender(title, sig=sig):
    index = indices[title]
    print(index)
    
    # get similarity scores
    sig_scores = list(enumerate(sig[index]))
    #sort by scores
    sig_scores = sorted(sig_scores, key=lambda x:x[1], reverse=True)
    sig_scores = sig_scores[1:6]
    
    #relate to book
    book_indices = [i[0] for i in sig_scores]
    
    return book_rating['itemID'].iloc[book_indices]

In [ ]:
recommender_df = recommender('Einfach zeichnen! Step by Step').to_frame()

In [ ]:
recommender_df

## Populate evaluation DF Part 3

In [ ]:
evaluation_df

In [ ]:
title_df = book_rating_B2.drop(['basket','order','main topic','subtopics','log_frequency','Grade', 'relevance_score', 'click','Frequency'], axis=1)
title_df

In [ ]:
submission_df_p3 = pd.merge(evaluation_df, title_df, on=["itemID"]) 
# submission_df_p1 = submission_df_p1.drop(['author','publisher','main topic','subtopics'], axis=1)
# submission_df_p1 = submission_df_p1.drop('index')
submission_df_p3.reset_index(drop=True, inplace=True)

In [ ]:
def rec3(submission_df_p3, num):
    recommended_books = []
    recommender_df = recommender(submission_df_p3['title']).to_frame()
    recommender_df = recommender_df.reset_index(drop=True)
    for ind,row in recommender_df.iterrows():
        recommended_books.append(row['itemID'])
        
    return recommended_books[num]

In [ ]:
recommend = ['rec1','rec2','rec3','rec4','rec5']
for i in range(0,5):
    key = recommend[i]
    submission_df_p3[key] = submission_df_p3.apply(rec3,num=i, axis=1)

In [ ]:
submission_df_p3 = submission_df_p3.drop(['title','parsedFeatureStr'], axis=1)

In [ ]:
submission_df_p3

In [ ]:
Export_csv_p3 = submission_df_p3.to_csv("submission_df_p3.csv", index=False)

## Combine CSV

In [ ]:
final_df_A = pd.read_csv("submission_df_p2.csv")
final_df_B = pd.read_csv("submission_df_p3.csv")

In [ ]:
final_df = pd.DataFrame()

final_df = final_df.append(final_df_A)
final_df = final_df.append(final_df_B)

In [ ]:
final_df.drop_duplicates(keep='first' ,inplace=True)
final_df

In [ ]:
Export_csv_p2 = final_df.to_csv("final_submission_df_p2.csv", index=False)

In [ ]:
import os 
import pandas as pd

final_submission = pd.DataFrame()

for file in os.listdir(os.getcwd()):
    if file.startswith('final'):
        print(file)
        final_submission = final_submission.append(pd.read_csv(file))

final_submission.to_csv('Uni_Asia_Pacific_1.csv', index=False, sep='|')